<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/model_build%5Cmodel_build_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2

In [2]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

Cloning into 'skin-disease-build'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 18517 (delta 71), reused 106 (delta 32), pack-reused 18361
Receiving objects: 100% (18517/18517), 1.61 GiB | 36.54 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Checking out files: 100% (19583/19583), done.


In [3]:
train_dir = r"/content/skin-disease-build/dataset/train"
test_dir=r"/content/skin-disease-build/dataset/test"
os.listdir(train_dir)

['Melanoma Skin Cancer Nevi and Moles',
 'Seborrheic Keratoses and other Benign Tumors',
 'Lupus and other Connective Tissue diseases',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Light Diseases and Disorders of Pigmentation',
 'Systemic Disease',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Warts Molluscum and other Viral Infections',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Atopic Dermatitis Photos',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Nail Fungus and other Nail Disease',
 'Eczema Photos',
 'Urticaria Hives',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Bullous Disease Photos',
 'Herpes HPV and other STDs Photos',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Exanthems and Drug Eruptions',
 'Acne and Rosacea Photos',
 'Tinea Ringworm Candidiasis and other Fungal Infections']

In [4]:
datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        validation_split = 0.2)

In [5]:
train_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'training')
valid_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'validation')

Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.


In [10]:
base_model = MobileNetV2(input_shape = (200, 200, 3),include_top=False,pooling='max')
model = Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(23, activation = 'softmax'))

In [11]:
base_model.trainable = False

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 23)                2967      
Total params: 2,424,919
Trainable params: 166,935
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
optimizer = Adam()

model.compile(optimizer = optimizer , loss = "sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])

In [14]:
epochs = 40

In [15]:
history = model.fit_generator(train_gen,epochs = epochs,
                             validation_data = valid_gen,
                             steps_per_epoch = train_gen.n//train_gen.batch_size,
                             validation_steps = valid_gen.n//valid_gen.batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
389/389 [==============================] - 682s 2s/step - loss: 4.0363 - sparse_categorical_accuracy: 0.0780 - val_loss: 3.0526 - val_sparse_categorical_accuracy: 0.0947
Epoch 2/40
389/389 [==============================] - 665s 2s/step - loss: 3.0432 - sparse_categorical_accuracy: 0.0895 - val_loss: 3.0106 - val_sparse_categorical_accuracy: 0.0905
Epoch 3/40
389/389 [==============================] - 669s 2s/step - loss: 3.0058 - sparse_categorical_accuracy: 0.0888 - val_loss: 2.9905 - val_sparse_categorical_accuracy: 0.0905
Epoch 4/40
389/389 [==============================] - 665s 2s/step - loss: 2.9873 - sparse_categorical_accuracy: 0.0909 - val_loss: 2.9814 - val_sparse_categorical_accuracy: 0.0905
Epoch 5/40
389/389 [==============================] - 671s 2s/step - loss: 2.9812 - sparse_categorical_accuracy: 0.0919 - val_loss: 2.9774 - val_sparse_categorical_accuracy: 0.0905
Epoch 6/40
299/389 [======================>.......] - ETA: 2:03 - loss: 2.9814 - sparse_categor

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['loss'],label = 'loss')
plt.plot(history.history['val_loss'],label = 'val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'],label = 'accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'],label = 'val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()